In [1]:
pip list

Package                      Version
---------------------------- ---------
absl-py                      1.4.0
asttokens                    2.0.5
astunparse                   1.6.3
backcall                     0.2.0
brotlipy                     0.7.0
cachetools                   5.3.0
certifi                      2022.12.7
cffi                         1.15.1
charset-normalizer           2.0.4
conda                        23.1.0
conda-package-handling       1.9.0
contourpy                    1.0.5
cryptography                 37.0.1
cycler                       0.11.0
debugpy                      1.5.1
decorator                    5.1.1
DeepXDE                      1.8.0
entrypoints                  0.4
executing                    0.8.3
flatbuffers                  23.1.21
fonttools                    4.25.0
gast                         0.4.0
google-auth                  2.16.0
google-auth-oauthlib         0.4.6
google-pasta                 0.2.0
grpcio                       1.51.1
h5p

In [1]:
import sys
import os
#dir_path = os.path.dirname(os.path.realpath(__file__))
#sys.path.append(dir_path)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import argparse
import scipy.io
import numpy as np
import deepxde as dde # version 0.11 or higher
from deepxde.backend import tf

Using backend: tensorflow.compat.v1

2023-02-16 14:33:33.117155: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 14:33:33.261201: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-16 14:33:33.261238: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-16 14:33:33.956387: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object 

Instructions for updating:
non-resource variables are not supported in the long term


2023-02-16 14:33:35.192792: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-16 14:33:35.192860: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cx3-1-29.cx3.hpc.ic.ac.uk): /proc/driver/nvidia/version does not exist


In [2]:
## PDE Parameters (initialized for 1D PINN)
input = 2 # network input size 
num_hidden_layers = 4 # number of hidden layers for NN 
hidden_layer_size = 32 # size of each hidden layers 
output = 2 # network input size 
        
## Training Parameters
num_domain = 20000 # number of training points within the domain
num_boundary = 1000 # number of training boundary condition points on the geometry boundary
num_test = 1000 # number of testing points within the domain
MAX_MODEL_INIT = 16 # maximum number of times allowed to initialize the model
MAX_LOSS = 4 # upper limit to the initialized loss
epochs_init = 15000 # number of epochs for training initial phase
epochs_main = 10000 # number of epochs for main training phase
lr = 0.0005 # learning rate

In [3]:
a = 0.01
b = 0.15
D = 0.1
k = 8
mu_1 = 0.2
mu_2 = 0.3
epsilon = 0.002

## Geometry Parameters
min_x = 0.1
max_x = 10            
min_t = 1
max_t = 50
spacing = 0.1

In [4]:
file_name = "data_1d_left.mat"
data = scipy.io.loadmat(file_name)
t, x, Vsav, Wsav = data["t"], data["x"], data["Vsav"], data["Wsav"]
X, T = np.meshgrid(x, t)
max_t = np.max(t)
max_x = np.max(x)        
X = X.reshape(-1, 1)
T = T.reshape(-1, 1)
V = Vsav.reshape(-1, 1)
W = Wsav.reshape(-1, 1)

In [5]:
observe_x = np.hstack((X,T))

In [6]:
def IC_func(observe_train, v_train):
        
        T_ic = observe_train[:,-1].reshape(-1,1)
        idx_init = np.where(np.isclose(T_ic,1))[0]
        v_init = v_train[idx_init]
        observe_init = observe_train[idx_init]
        return dde.PointSetBC(observe_init,v_init,component=0)

In [7]:
def BC_func(dim, geomtime):
    bc = dde.NeumannBC(geomtime, lambda x:  np.zeros((len(x), 1)), lambda _, on_boundary: on_boundary, component=0)
    return bc

In [8]:
    
## Split data to train and test
observe_train, observe_test, v_train, v_test, w_train, w_test = train_test_split(observe_x,V,W,test_size=0.8)    

In [9]:
geom = dde.geometry.Interval(min_x, max_x)
timedomain = dde.geometry.TimeDomain(min_t, max_t)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)  

In [10]:
## Define Boundary Conditions
bc = BC_func(1, geomtime)

## Define Initial Conditions
ic = IC_func(observe_train, v_train)
    
## Model observed data
observe_v = dde.PointSetBC(observe_train, v_train, component=0)
input_data = [bc, ic, observe_v]

In [11]:
net = dde.maps.FNN([2] + [hidden_layer_size] * num_hidden_layers + [2], "tanh", "Glorot uniform")


In [12]:
def pde_1D(x, y):
    
    V, W = y[:, 0:1], y[:, 1:2]
    dv_dt = dde.grad.jacobian(y, x, i=0, j=1)
    dv_dxx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    dw_dt = dde.grad.jacobian(y, x, i=1, j=1)
    ## Coupled PDE+ODE Equations
    eq_a = dv_dt -  D*dv_dxx + k*V*(V-a)*(V-1) +W*V 
    eq_b = dw_dt -  (epsilon + (mu_1*W)/(mu_2+V))*(-W -k*V*(V-b-1))
    return [eq_a, eq_b]

In [13]:
pde = pde_1D

In [14]:
pde_data = dde.data.TimePDE(geomtime, pde, input_data,
                            num_domain = num_domain, 
                            num_boundary=num_boundary, 
                            anchors=observe_train,
                            num_test=num_test)  

In [28]:
print(num_domain)
print(num_boundary)
print(observe_train.shape)
print(num_test)
print(type(pde))
print(observe_train[0].size)
print(observe_train[1].size)
print(len(input_data))
print(input_data[1])
print(bc)

20000
1000
(2800, 2)
1000
<class 'function'>
2
2
3


In [15]:
model = dde.Model(pde_data, net)
model.compile("adam", lr=lr)

Compiling model...
Building feed-forward neural network...
'build' took 0.066638 s



/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:114: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
2023-02-14 15:05:22.167869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'compile' took 0.603212 s



In [16]:
def stable_init(model):
    ## Stabalize initialization process by capping the losses
    losshistory, _ = model.train(epochs=1)
    initial_loss = max(losshistory.loss_train[0])
    num_init = 1
    while initial_loss>MAX_LOSS or np.isnan(initial_loss):
        num_init += 1
        model = dde.Model(pde_data, net)
        model.compile("adam", lr=lr)
        losshistory, _ = model.train(epochs=1)
        initial_loss = max(losshistory.loss_train[0])
        if num_init > MAX_MODEL_INIT:
            raise ValueError('Model initialization phase exceeded the allowed limit')
    return 0

In [17]:
out_path = 'test.txt'

In [18]:
def train_1_phase(model, out_path):
    
    losshistory, train_state = model.train(epochs=epochs_main, model_save_path = out_path)
    return losshistory, train_state

In [19]:
def train(model, out_path):
        
        ## Stabalize initialization process by capping the losses
        stable_init(model)
      
     
        losshistory, train_state = train_1_phase(model, out_path)
                   
        return model, losshistory, train_state

In [20]:
model, losshistory, train_state = train(model, out_path)

Initializing variables...


2023-02-14 15:05:37.194633: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled


Training model...

Step      Train loss                                            Test loss                                             Test metric
0         [5.80e+00, 2.84e+03, 6.89e-04, 9.47e-02, 6.41e-01]    [5.39e+00, 4.94e+01, 6.89e-04, 9.47e-02, 6.41e-01]    []  
1         [1.32e+01, 2.37e+03, 6.03e-04, 1.39e-01, 7.92e-01]    [1.25e+01, 5.49e+01, 6.03e-04, 1.39e-01, 7.92e-01]    []  

Best model at step 1:
  train loss: 2.39e+03
  test loss: 6.84e+01
  test metric: []

'train' took 1.041585 s

Compiling model...
'compile' took 0.458314 s

Initializing variables...
Training model...

Step      Train loss                                            Test loss                                             Test metric
0         [1.71e-02, 2.15e-01, 5.24e-04, 1.24e-03, 2.83e-01]    [1.63e-02, 1.97e-01, 5.24e-04, 1.24e-03, 2.83e-01]    []  
1         [2.49e-02, 4.35e-02, 4.67e-04, 1.47e-03, 2.33e-01]    [2.52e-02, 4.29e-02, 4.67e-04, 1.47e-03, 2.33e-01]    []  

Best model at step 1:
  t

2023-02-14 15:05:39.293945: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_4/bias/Adam_1/Assign' id:1435 op device:{requested: '', assigned: ''} def:{{{node dense_4/bias/Adam_1/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@dense_4/bias"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](dense_4/bias/Adam_1, dense_4/bias/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-02-14 15:05:39.374695: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_4/bias/Adam_3/Assign' id:2696 op device:{requested: '', assigned: ''} def:{{{node dense_4/bias/Adam_3/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@dense_4/bias"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](dense_4/bias/Adam_3, dense_4/bias/Adam_3/Initializer/zeros)}}' was changed by setting

1000      [7.58e+01, 5.73e-01, 1.02e-03, 5.68e-01, 1.26e+00]    [7.39e+01, 5.73e-01, 1.02e-03, 5.68e-01, 1.26e+00]    []  
2000      [7.58e+01, 5.74e-01, 1.02e-03, 5.68e-01, 1.26e+00]    [7.39e+01, 5.74e-01, 1.02e-03, 5.68e-01, 1.26e+00]    []  
3000      [7.58e+01, 5.75e-01, 1.02e-03, 5.68e-01, 1.26e+00]    [7.39e+01, 5.75e-01, 1.02e-03, 5.68e-01, 1.26e+00]    []  
4000      [7.58e+01, 5.76e-01, 1.02e-03, 5.68e-01, 1.26e+00]    [7.39e+01, 5.76e-01, 1.02e-03, 5.68e-01, 1.26e+00]    []  
5000      [7.58e+01, 5.79e-01, 1.02e-03, 5.68e-01, 1.26e+00]    [7.38e+01, 5.79e-01, 1.02e-03, 5.68e-01, 1.26e+00]    []  
6000      [7.58e+01, 5.84e-01, 1.02e-03, 5.68e-01, 1.26e+00]    [7.38e+01, 5.84e-01, 1.02e-03, 5.68e-01, 1.26e+00]    []  
7000      [7.58e+01, 5.93e-01, 1.02e-03, 5.68e-01, 1.26e+00]    [7.38e+01, 5.93e-01, 1.02e-03, 5.68e-01, 1.26e+00]    []  
8000      [7.58e+01, 6.06e-01, 1.02e-03, 5.68e-01, 1.26e+00]    [7.38e+01, 6.06e-01, 1.02e-03, 5.68e-01, 1.26e+00]    []  
9000      [7.57e

In [21]:
pred = model.predict(observe_test)
v_pred = pred[:,0:1]
rmse_v = np.sqrt(np.square(v_pred - v_test).mean())

In [22]:
print("V rMSE for test data:", rmse_v)

V rMSE for test data: 1.1218949013362933


In [23]:
print(num_domain)
print(num_boundary)
print(observe_train.shape)
print(num_test)

20000
1000
(2800, 2)
1000
